# DHT11 Data Logging with Raspberry Pi
In this notebook, we demonstrate how to read temperature and humidity data from a **DHT11** sensor on a Raspberry Pi using the Adafruit CircuitPython DHT library, and then log the data to a CSV file.

## What You Will Learn
1. How to import the required libraries for handling time, board pins, DHT sensors, and CSV file operations.
2. How to initialize the DHT11 sensor on a specific GPIO pin.
3. How to set up a CSV file for data logging.
4. How to continuously read data and handle common errors from DHT sensors.
5. How to append data to a CSV file in real time.

> **Note:** This notebook assumes that you have already installed `adafruit-circuitpython-dht` on your Raspberry Pi. Make sure your DHT sensor is wired to the correct GPIO pin. You do not need I2C or SPI for DHT sensors; they use a single-wire protocol.

## 1. Import Required Libraries
We need:
- **time**: to handle delays and timestamp formatting.
- **board**: to access GPIO pin definitions on the Raspberry Pi.
- **adafruit_dht**: CircuitPython library for reading DHT sensors.
- **csv**: to write data into a CSV file.

In [ ]:
import time
import board
import adafruit_dht
import csv

print("Libraries imported successfully!")

## 2. Initialize the DHT Sensor
You can choose either a **DHT11** or **DHT22** sensor. Here, we uncomment the one we want to use. We also specify which GPIO pin is used.

> **Important**: The DHT sensor libraries typically expect you to refer to pins using the `board` module (for example, `board.D4`), which corresponds to BCM GPIO 4 on the Raspberry Pi.

In [ ]:
# Uncomment this line if you have a DHT22 sensor:
# sensor = adafruit_dht.DHT22(board.D4)

# Uncomment this line if you have a DHT11 sensor:
sensor = adafruit_dht.DHT11(board.D4)

print("DHT sensor initialized.")

## 3. Prepare the CSV File
We specify the path to our CSV file (for example, `./sensor_data.csv`) and write a header row containing:
1. Time (timestamp)
2. Temperature in Celsius (°C)
3. Temperature in Fahrenheit (°F)
4. Humidity in percentage (%)

The `with open` context manager will ensure the file is properly closed after writing.

In [ ]:
csv_file = "./sensor_data.csv"

# Open the file in 'write' mode to create or overwrite it, then write the CSV header
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Time", "Temperature (°C)", "Temperature (°F)", "Humidity (%)"])

print(f"Data will be saved to {csv_file}")

## 4. Continuously Read and Log Data
The main loop performs these steps:
1. Get the current time in a readable format.
2. Read the **temperature** in Celsius from the DHT sensor.
3. Convert the temperature to Fahrenheit.
4. Read the **humidity**.
5. Print the data to the console.
6. Append the data to the CSV file.
7. Sleep for a few seconds and repeat.

### Error Handling
- DHT sensors can be finicky, so `RuntimeError` can occur if the reading fails.
- We catch these errors, wait a couple of seconds, and continue.
- For any other Exception, we exit the sensor properly and then raise the error.

In [ ]:
print("Starting data collection. Press Stop or interrupt the kernel to exit.")

while True:
    try:
        # 1. Get the current time
        measurement_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

        # 2. Read temperature in Celsius
        temperature_c = sensor.temperature

        # 3. Convert temperature to Fahrenheit
        temperature_f = temperature_c * (9.0 / 5.0) + 32.0

        # 4. Read humidity
        humidity = sensor.humidity

        # 5. Print to console
        print(f"[{measurement_time}] Temp={temperature_c:.1f}ºC, "
              f"Temp={temperature_f:.1f}ºF, "
              f"Humidity={humidity:.1f}%")

        # 6. Append values to CSV
        with open(csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([measurement_time, temperature_c, temperature_f, humidity])

    except RuntimeError as error:
        # This happens occasionally with DHT sensors; retry after a short pause
        print(f"RuntimeError encountered: {error}")
        time.sleep(2.0)
        continue
    except Exception as error:
        # Exit the sensor and re-raise any unexpected error
        sensor.exit()
        raise error

    # 7. Sleep for 3 seconds before the next reading
    time.sleep(3.0)

## 5. Summary
In this notebook, we:
1. Imported the necessary libraries: `time`, `board`, `adafruit_dht`, and `csv`.
2. Initialized a **DHT11** sensor on GPIO 4 (`board.D4`).
3. Created (or overwrote) a CSV file named `sensor_data.csv` and wrote a header row.
4. Started a loop to read data from the DHT11 sensor, print it to the console, and append each reading to the CSV file.
5. Handled common reading errors (e.g., `RuntimeError`) by retrying, and performed a clean exit in case of unexpected exceptions.

### Next Steps
- Swap the sensor initialization for **DHT22** if you have that model.
- Experiment with different logging intervals.
- Visualize the data by reading the CSV file and plotting temperature/humidity over time.
- Integrate with a dashboard or web server to view readings in real time.

Happy hacking!